In [6]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import pymupdf, fitz

c:\Users\Maxim\Desktop\hw\X5\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import pandas as pd

In [8]:
def get_text_with_links(document):
    result_text = ""
    
    for page_num in range(len(document)):
        page = document.load_page(page_num)

        # Получаем слова и их координаты
        words = page.get_text("words")
        links = page.get_links()

        # Временный словарь для отслеживания последнего индекса слова для каждого URL
        last_occurrence_with_url = {}

        wc_map = {tuple(word[:4]): word[4] for word in words}
        
        for link in links:
            link_rect = fitz.Rect(link["from"])
            uri = link.get('uri', 'Нет ссылки')
            
            for i, word in enumerate(words):
                text = word[4]   # текст слова
                word_rect = fitz.Rect(word[:4])

                # Проверка на пересечение координат
                if word_rect.intersects(link_rect):
                    last_occurrence_with_url[uri] = (i, tuple(word[:4]))

        # Заменяем текст в конечном словаре ссылками на последних вхождениях
        for uri, (index, rect) in last_occurrence_with_url.items():
            wc_map[rect] = f'{wc_map[rect]}:"{uri}"'

        # Сортировка слов по их координатам для правильного порядка
        sorted_words = sorted(wc_map.items(), key=lambda x: (x[0][1], x[0][0]))

        for item in sorted_words:
            result_text += item[1] + " "
    
    return result_text.strip()  # добавляем strip для удаления лишних пробелов в конце

In [9]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large', device_map="cuda")
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large', device_map="cuda")

In [10]:
df = pd.read_csv('./interns_preprocessed/interns_preprocessed.csv').dropna().reset_index(drop=True)

In [11]:
df['text']=None
df['embedding']=None

In [12]:
for i, pdf_file in enumerate(tqdm(df['Резюме'])):
    pdf_file = pdf_file.replace('\\', '\\\\')
    #print(pdf_file)
    try:
        with pymupdf.open(pdf_file) as doc:
            text = get_text_with_links(doc)
    except:
        print('битый файл')
        continue
    
    batch_dict = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt').to('cuda')
    embeddings = average_pool(model(**batch_dict).last_hidden_state, batch_dict['attention_mask'])
    df.loc[i, ['text']] = [text]
    df.loc[i, ['embedding']] = [embeddings.tolist()]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 311/311 [00:11<00:00, 26.16it/s]


In [13]:
df['label'] = df['Hire status']

In [17]:
from datasets import Dataset

In [19]:
dataset = Dataset.from_pandas(df[['text', 'label']])

In [23]:
dataset = dataset.class_encode_column("label")

Casting to class labels: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 311/311 [00:00<00:00, 101884.60 examples/s]


In [26]:
dataset.train_test_split(test_size=0.1, seed=42, stratify_by_column='label').save_to_disk('splitted_dataset')

Saving the dataset (1/1 shards): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 7544.14 examples/s]
